In [1]:
from market_growth_analysis.web_scraping import process_sector, scrape_company_data
import concurrent
import tqdm

from bs4 import BeautifulSoup
import requests

In [2]:
URL = "https://www.macrotrends.net/stocks/research"
page = requests.get(URL)


soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all("a", href= lambda text: "/stocks/sector/" in text.lower())
sector_dict = dict()

for result in results:
    sector_dict[result.text] = ("".join(("https://www.macrotrends.net", result["href"]))).strip()

with concurrent.futures.ThreadPoolExecutor() as executor:
    print("Retrieving income statement")
    financial_url = "income-statement"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, f"{financial_url}?freq=A", executor)

    print("Retrieving cash flow statements")
    financial_url = "cash-flow-statement"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, f"{financial_url}?freq=A", executor)

    print("Retrieving balance sheet")
    financial_url = "balance-sheet"
    for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
        process_sector(sector, sector_dict, f"{financial_url}?freq=A", executor)

Retrieving income statement


Sectors:   0%|          | 0/17 [00:00<?, ?it/s]


TypeError: process_sector() takes 3 positional arguments but 4 were given